### Loading the dataset

In [6]:
!pip uninstall jedi
from datasets import load_dataset
from pathlib import Path

In [13]:
# reading the jsonl dataset
data_dir = Path("../joes_transgression_ambiguity_project/data/annotated_tweets/ta_tweets.jsonl")

import pandas as pd    
jsonObj = pd.read_json(path_or_buf=data_dir, lines=True)



In [42]:
jsonObj.head()["answer"][2]

'reject'

In [18]:
jsonObj.columns

Index(['text', 'meta', '_input_hash', '_task_hash', 'label', 'score',
       'priority', 'spans', '_session_id', '_view_id', 'answer'],
      dtype='object')

### Huggingface dataset loading script
can be used later to publish the dataset on huggingface hub for better outreach

In [41]:
# coding=utf-8
# Copyright 2020 The HuggingFace Datasets Authors and the current dataset script contributor.
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     http://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.
"""

This is script loads the annotations for the moral transgression - twitter condemnation project.
The datasets is aimed at building two classifiers. 
1. detecting condemnation in a tweet
2. estimating the severity of condemnation

"""


import csv
import json
import os

import datasets


# TODO: Add BibTeX citation
# Find for instance the citation on arxiv or on the dataset repo/website
_CITATION = """\
@InProceedings{huggingface:dataset,
title = {A great new dataset},
author={huggingface, Inc.
},
year={2020}
}
"""

# TODO: Add description of the dataset here
# You can copy an official description
_DESCRIPTION = """\
The datasets is aimed at building two classifiers. 
1. detecting condemnation in a tweet
2. estimating the severity of condemnation
"""

# TODO: Add a link to an official homepage for the dataset here
_HOMEPAGE = ""

# TODO: Add the licence for the dataset here if you can find it
_LICENSE = ""

# TODO: Add link to the official dataset URLs here
# The HuggingFace dataset library don't host the datasets but only point to the original files
# This can be an arbitrary nested dict/list of URLs (see below in `_split_generators` method)
_URLs = {
    'first_domain': "../joes_transgression_ambiguity_project/data/annotated_tweets/ta_tweets.jsonl",
    'second_domain': "https://huggingface.co/great-new-dataset-second_domain.zip",
}


# TODO: Name of the dataset usually match the script name with CamelCase instead of snake_case
class CondemnationDataset(datasets.GeneratorBasedBuilder):
    """TODO: Short description of my dataset."""

    VERSION = datasets.Version("1.1.0")

    # This is an example of a dataset with multiple configurations.
    # If you don't want/need to define several sub-sets in your dataset,
    # just remove the BUILDER_CONFIG_CLASS and the BUILDER_CONFIGS attributes.

    # If you need to make complex sub-parts in the datasets with configurable options
    # You can create your own builder configuration class to store attribute, inheriting from datasets.BuilderConfig
    # BUILDER_CONFIG_CLASS = MyBuilderConfig

    # You will be able to load one or the other configurations in the following list with
    # data = datasets.load_dataset('my_dataset', 'first_domain')
    # data = datasets.load_dataset('my_dataset', 'second_domain')
    BUILDER_CONFIGS = [
        datasets.BuilderConfig(name="first_domain", version=VERSION, description="This part of my dataset covers a first domain"),
        datasets.BuilderConfig(name="second_domain", version=VERSION, description="This part of my dataset covers a second domain"),
    ]

    DEFAULT_CONFIG_NAME = "first_domain"  # It's not mandatory to have a default configuration. Just use one if it make sense.

    def _info(self):
        # TODO: This method specifies the datasets.DatasetInfo object which contains informations and typings for the dataset
        if self.config.name == "first_domain":  # This is the name of the configuration selected in BUILDER_CONFIGS above
            #'text', 'meta', '_input_hash', '_task_hash', 'label', 'score', 'priority', 'spans', '_session_id', '_view_id', 'answer'
            features = datasets.Features(
                {
                    "text": datasets.Value("string"),
                    "meta": datasets.Value("dict"),
                    "_input_hash": datasets.Value("numpy.int64"),
                    "_task_hash": datasets.Value("numpy.int64"),
                    "label": datasets.Value("string"),
                    "score": datasets.Value("numpy.float64"),
                    "priority": datasets.Value("numpy.float64"),
                    "spans": datasets.Value("list"),
                    "_session_id": dataset.Value("string"),
                    "_view_id": dataset.Value("string"),
                    "answer": dataset.Value("string")
                    # These are the features of your dataset like images, labels ...
                }
            )
        else:  # This is an example to show how to have different features for "first_domain" and "second_domain"
            features = datasets.Features(
                {
                    "sentence": datasets.Value("string"),
                    "option2": datasets.Value("string"),
                    "second_domain_answer": datasets.Value("string")
                    # These are the features of your dataset like images, labels ...
                }
            )
        return datasets.DatasetInfo(
            # This is the description that will appear on the datasets page.
            description=_DESCRIPTION,
            # This defines the different columns of the dataset and their types
            features=features,  # Here we define them above because they are different between the two configurations
            # If there's a common (input, target) tuple from the features,
            # specify them here. They'll be used if as_supervised=True in
            # builder.as_dataset.
            supervised_keys=None,
            # Homepage of the dataset for documentation
            homepage=_HOMEPAGE,
            # License for the dataset if available
            license=_LICENSE,
            # Citation for the dataset
            citation=_CITATION,
        )

    def _split_generators(self, dl_manager):
        """Returns SplitGenerators."""
        # TODO: This method is tasked with downloading/extracting the data and defining the splits depending on the configuration
        # If several configurations are possible (listed in BUILDER_CONFIGS), the configuration selected by the user is in self.config.name

        # dl_manager is a datasets.download.DownloadManager that can be used to download and extract URLs
        # It can accept any type or nested list/dict and will give back the same structure with the url replaced with path to local files.
        # By default the archives will be extracted and a path to a cached folder where they are extracted is returned instead of the archive
        my_urls = _URLs[self.config.name]
        data_dir = dl_manager.download_and_extract(my_urls)
        return [
            datasets.SplitGenerator(
                name=datasets.Split.TRAIN,
                # These kwargs will be passed to _generate_examples
                gen_kwargs={
                    "filepath": os.path.join(data_dir, "train.jsonl"),
                    "split": "train",
                },
            ),
            datasets.SplitGenerator(
                name=datasets.Split.TEST,
                # These kwargs will be passed to _generate_examples
                gen_kwargs={
                    "filepath": os.path.join(data_dir, "test.jsonl"),
                    "split": "test"
                },
            ),
            datasets.SplitGenerator(
                name=datasets.Split.VALIDATION,
                # These kwargs will be passed to _generate_examples
                gen_kwargs={
                    "filepath": os.path.join(data_dir, "dev.jsonl"),
                    "split": "dev",
                },
            ),
        ]

    def _generate_examples(
        self, filepath, split  # method parameters are unpacked from `gen_kwargs` as given in `_split_generators`
    ):
        """ Yields examples as (key, example) tuples. """
        # This method handles input defined in _split_generators to yield (key, example) tuples from the dataset.
        # The `key` is here for legacy reason (tfds) and is not important in itself.

        with open(filepath, encoding="utf-8") as f:
            for id_, row in enumerate(f):
                data = json.loads(row)
                if self.config.name == "first_domain":
                    yield id_, {
                        "sentence": data["sentence"],
                        "option1": data["option1"],
                        "answer": "" if split == "test" else data["answer"],
                    }
                else:
                    yield id_, {
                        "sentence": data["sentence"],
                        "option2": data["option2"],
                        "second_domain_answer": "" if split == "test" else data["second_domain_answer"],
                    }


In [47]:
dataset = load_dataset("jsonl", data_files= ../joes_transgression_ambiguity_project/data/annotated_tweets/ta_tweets.jsonl)

SyntaxError: invalid syntax (<ipython-input-47-9df887cb1a05>, line 1)

### Testing the dataset loading script

In [57]:
from datasets import load_dataset
dataset = load_dataset("./dataset_loading_script.py", name="first_domain", data_files="../joes_transgression_ambiguity_project/data/annotated_tweets/ta_tweets.jsonl")

Using custom data configuration first_domain-3bcd13f68545b6d6


0 examples [00:00, ? examples/s]

OSError: Cannot find data file. 
Original error:
[Errno 20] Not a directory: '../joes_transgression_ambiguity_project/data/annotated_tweets/ta_tweets.jsonl/train.jsonl'